# i.MXRT118x Debug Authentication example
I.MX RT118x offers the Debug Authentication Protocol (DAP) as a mechanism to authenticate the debugger (an external entity) which has the credentials approved by the product manufacturer before granting the debug access to the device. This example demonstrates process how to establish Debug Authentication protocol.

The process could be split into following steps:

## Introduction
The fundamental principles of debugging, which require access to the system state and system information, conflict with the principles of security, which require the restriction of access to assets. Thus, many products disable debug access completely before deploying the product. To address these challenges, the chip offers a debug authentication protocol as a mechanism to authenticate the debugger (an external entity) has the credentials approved by the product manufacturer before granting debug access to the device.
The debug authentication is a challenge-response scheme and assures that only the debugger in possession of the required debug credentials can successfully authenticate over the debug interface and access restricted parts of the device. 

The protocol is divided into steps as described below:
1. The debugger initiates the Debug Mailbox message exchange by setting the CSW[RESYNCH_REQ] bit and
CSW[CHIP_RESET_REQ] bit of DM-AP.
2. The debugger waits (minimum 30 ms) for the devices to restart and enter debug mailbox request handling loop.
3. The debugger sends Debug Authentication Start command (command code 10h) to the device.
4. The device responds back with Debug Authentication Challenge (DAC) packet based on the debug access rights preconfigured in CMPA fields, which are collectively referred as Device Credential Constraints Configuration (DCFG_CC).
The response packet also contains a 32 bytes random challenge vector.
5. The debugger responds to the challenge with a Debug Authentication Response (DAR) message by using an
appropriate debug certificate, matching the device identifier in the DAC. The DAR packet contains the debug access
permission certificate, also referred as Debug Credential (DC), and a cryptographic signature binding the DC and the
challenge vector provided in the DAC.
6. The device on receiving the DAR, validates the contents by verifying the cryptographic signature of the message using
the debugger's public key present in the embedded the Debug Credential (DC). On successful validation of DAR, the
device enables access to the debug domains permitted in the DC

![debug_authentication_flow](img/debug_authentication_flow.PNG "debug_authentication_flow.PNG")

### Prepare the Environment

In [1]:
%run ../init_notebook.ipynb

import os
import pprint

pp = pprint.PrettyPrinter(indent=4)

WORKSPACE = "workspace/"  # change this to path to your workspace
INPUTS = "inputs/"
VERBOSITY = (
    "-v"  # verbosity of commands, might be -v or -vv for debug or blank for no additional info
)

DBGMAILBOX_TEMPLATE = (
    WORKSPACE + "nxpdebugmbox_template.yaml"
)  # Template for debug mailbox configuration
DBGMAILBOX_DC_TEMPLATE = (
    WORKSPACE + "nxpdebugmbox_dc_template.yaml"
)  # Template for debug credential file configuration
DBGMAILBOX_CONFIG = (
    INPUTS + "nxpdebugmbox_rt118x.yaml"
)  # yaml file with configuration used in this example
DBGMAILBOX_DC_CONFIG = (
    INPUTS + "nxpdebugmbox_dc_rt118x.yaml"
)  # yaml file with configuration used in this example for debug credential file

# DEBUGGER_PROBE = "pyocd" #onboard debugger on EVK-RT118x rev C
DEBUGGER_PROBE = "pyocd"
# Only one probe can be connected to the board.

FAMILY = "-f mimxrt1189"

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


### EVK Board Overview
The following picture describes connector placement of RT1180 EVK.
![Signing key and used SRK definition](img/evk-1.png "signing key")

The Enablement process could be split into following steps:

### 1. Generate Keys
The AHAB is using asymmetric algorithm for image authentication which requires keys generated according to PKI for operation. One of the generated private key must be also used to sing Debug Credential file. Another key pair which must be generated for this example is Debug credential keys (DCKPriv/Pub). The example uses pre-generated ECC-256 SRK and DCK keys. To generate your own keys please refer to [How-to-get-keys-using-nxpcrypto](../general/crypto.ipynb) example.

### 2. Create Debug Credential file.

#### 2.1 Create a template file

In [2]:
# Generate template
%! nxpdebugmbox $FAMILY dat dc get-template --force -o $DBGMAILBOX_DC_TEMPLATE

nxpdebugmbox -f mimxrt1189 dat dc get-template --force -o workspace/nxpdebugmbox_dc_template.yaml 
The Debug Credentials template for mimxrt1189 has been saved into             workspace\nxpdebugmbox_dc_template.yaml YAML file


#### 2.2 Update template to implement desired functionality and add reference to specific keys.
This yaml file must be used to generate Debug Credential file. The OEM is must do it as posses the RoT private keys. Field technician must provide his DCK public key to OEM. The resulting yaml file is available in the "inputs" folder. The figures below illustrates which entries were changed.


I. The template file uses zeroized UUID. To create a debug credential file for specific par run the "get-uuid" command listed below and replace to zeros at the UUID field by the UUID value returned by the command in the yaml file.
Vendor usage and cc beacon is not demonstrated in this example.

![Key modification](img/debug_uath-1.png "key modification")

Connect debug probe into the connector.
To read UUID the following command has to be activated (# must be removed).

In [3]:
# Check the nxp debug mailbox tool
# %! nxpdebugmbox $FAMILY -i $DEBUGGER_PROBE tool get-uuid



II. Keys must be modified in the file. This example uses pregenerated keys. To Generate custom key, please refer to the example: [How-to-get-keys-using-nxpcrypto](../general/get_keys.ipynb). 
Reference to RoT public keys, Signing RoT private key (index 1), and DCK public key must be modified in the template file.


![Key modification](img/debug_uath-2.png "key modification")

In [4]:
# Create Debug Credential file

# Debug Credential file as output of
DC_FILE_OUT = WORKSPACE + "rt118x_256.dc"

# Assert os.path.exists(DBGMAILBOX_CONFIG)
assert os.path.isfile(DBGMAILBOX_DC_CONFIG)

%! nxpdebugmbox $FAMILY -v dat dc export -c $DBGMAILBOX_DC_CONFIG -o $DC_FILE_OUT --force

# Assert os.path.exists(DC_FILE_OUT)
assert os.path.isfile(DC_FILE_OUT)

nxpdebugmbox -f mimxrt1189 -v dat dc export -c inputs/nxpdebugmbox_dc_rt118x.yaml -o workspace/rt118x_256.dc --force 
INFO:spsdk.apps.nxpdebugmbox:Loading configuration from yml file...
INFO:spsdk.dat.debug_credential:Protocol version not defined. The version 2.0 has been determined from RoT public key
INFO:spsdk.apps.nxpdebugmbox:Creating ECC debug credential object...
RKTH: cb2cc774b2dcec92c840eca0646b78f8d3661d3a43ed265a490a13aca75e190a
INFO:spsdk.apps.nxpdebugmbox:Saving the debug credential to a file: workspace/rt118x_256.dc
Creating Debug credential file succeeded


#### 3 Run debug authentication procedure

To proper run debug authentication procedure a tiny configuration file is needed that specify the data from in-field technician. The resulting yaml file is available in the "inputs" folder. The figures below illustrates which entries were changed. 

#### 3.1 Create a template file

In [5]:
# Generate template
%! nxpdebugmbox $FAMILY dat get-template --force -o $DBGMAILBOX_TEMPLATE

nxpdebugmbox -f mimxrt1189 dat get-template --force -o workspace/nxpdebugmbox_template.yaml 
Creating workspace\nxpdebugmbox_template.yaml template file.


#### 3.2 Update template to implement desired functionality and add reference to specific keys.
This yaml file must be used to run Debug Authentication process (there could be different configuration for different devices). In our case we should do just two changes in template configuration file. The resulting yaml file is available in the "inputs" folder. The figures below illustrates which entries were changed.

1: Provide valid path to debug credential file:

![Changed the debug credential path](img/dc_path_change.png)

2: Provide path to used Debug credential file private key (Must match to public key stored in debug credential file signed by OEM root keys). In our case we don't use a signature provider, so the configuration of signature provider is erased.

![Change path to DCK private key and erase signature provider.png](img/dck_private_kay_path.png)

This changes are enough to do proper debug authentication procedure on mimxrt1189 device.


In [6]:
# Assert os.path.exists(DCK_KEY_PRIV)
assert os.path.isfile(DBGMAILBOX_CONFIG)

# Unlock the Debug Authentication
%! nxpdebugmbox $FAMILY -i $DEBUGGER_PROBE -v dat auth -c $DBGMAILBOX_CONFIG

nxpdebugmbox -f mimxrt1189 -i pyocd -v dat auth -c inputs/nxpdebugmbox_rt118x.yaml 
INFO:spsdk.apps.nxpdebugmbox:Starting Debug Authentication
  #   Interface   Id          Description                 
----------------------------------------------------------
  0   PyOCD       851002903   Segger J-Link Compact PLUS  
INFO:spsdk.debuggers.debug_probe_pyocd:PyOCD connected via J-Link Compact PLUS probe.
INFO:spsdk.apps.nxpdebugmbox:DAC: 
Version                : Version 2.0
SOCC                   : 0x5254049C
UUID                   : C8BDB7562AFA425B9528F5D63EB4125E
CC_VU                  : 0
ROTID_rkh_revocation   : 00000000
ROTID_rkth_hash        : cb2cc774b2dcec92c840eca0646b78f8d3661d3a43ed265a490a13aca75e190a
CC_soc_pinned          : 00FFFFF0
CC_soc_default         : 00000000
Challenge              : 2503a87dc8636eef9c488fa23347b8c0bf9d8894dfb908cbdd7a047ba830c857

INFO:spsdk.apps.nxpdebugmbox:DAR:
DAC:
Version                : Version 2.0
SOCC                   : 0x5254049C
UUID  

In [7]:
# Test if debug access works
# Do not call if you want to debug in IDE
%! nxpdebugmbox $FAMILY -i $DEBUGGER_PROBE mem-tool test-connection

nxpdebugmbox -f mimxrt1189 -i pyocd mem-tool test-connection 
  #   Interface   Id          Description                 
----------------------------------------------------------
  0   PyOCD       851002903   Segger J-Link Compact PLUS  
The device is accessible for debugging.
